In [1]:
# Librerias

# Manipulacion 
import pandas as pd 
import geopandas as gpd

# Earth engine (descarga de imagenes)
import ee 

# Mapas interactivos
import folium

# Plot
from IPython.display import Image

In [4]:
# Autentificar Earth engine
# ee.Authenticate() # Esto solo debe ser ejecutado la primera vez y cada vez que el token expira

ee.Initialize()


Successfully saved authorization token.


In [5]:
# Definir area de estudio

# Coordenadas
lat = 21.8833333
lon = -102.3

# Punto de interes
poi = ee.Geometry.Point(lon, lat)

# Periodo de tiempo
start_date = "2021-11-01"
end_date = "2022-11-15"

In [14]:
# Descarga de imagenes Landsat 9
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(poi)\
    .filterDate(start_date, end_date)

# Cantidad de imagenes obtenidas
print("Imagenes totales:", sentinel.size().getInfo())

Imagenes totales: 147


In [15]:
# Informacion de la primera imagen
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32613',
   'crs_transform': [60, 0, 699960, 0, -60, 2500020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [8]:
# Nubosidad
print(sentinel.first().get("CLOUD_COVER").getInfo())

None


In [32]:
# Fechas
print(ee.Date(sentinel.first().get("system:time_start")).getInfo())

{'type': 'Date', 'value': 1635961068158}


In [11]:
# Bandas 
sentinel.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [12]:
# Parametros para preprocesamiento
parameters = {
    "min":7000,
    "max":16000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B2", "B3", "B24"] # (R, G, B)
}

In [13]:
# Imagenes en lista
sentinel_list = sentinel.toList(sentinel.size())

# Funcion para mostrar cada imagen
def display_img(ids, parameters, ndvi_display = False):

    for i in ids:
    
        # Fecha de captura
        date = ee.Image(sentinel_list.get(i)).get("DATE_ACQUIRED").getInfo()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUD_COVER").getInfo()
    
        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud)

        if ndvi_display == True:
            display(Image(url = ee.Image(sentinel_list.get(i)).normalizedDifference(["SR_B5", "SR_B4"]).getThumbURL(parameters)))

        else:
            # Plot
            display(Image(url = ee.Image(sentinel_list.get(i)).getThumbURL(parameters)))
        
# Funcion para crear data frame con info de las imagenes
def img_info(ids):
    data = []
    for i in ids:

        # Fecha de captura
        date = ee.Image(sentinel_list.get(i)).get("DATE_ACQUIRED").getInfo()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUD_COVER").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud)
        
        image_data = [i, date, cloud]
        data.append(image_data)
    
    return data

NameError: name 'sentinel' is not defined

In [ ]:
# DF de informacion
sentinel_df = pd.DataFrame(img_info(range(sentinel.size().getInfo())), columns = ["img_id", "date", "cloud_cover"])

In [ ]:
# Imagenes recolectadas
display_img(sentinel_df.img_id, parameters = parameters)

In [ ]:
# Data frame con  informacion de las imagenes
sentinel_df.head() # Con esto se puede filtrar por la nubosidad, la cual parece ser buena por debajo de 10

## Filtrado de imagenes

In [ ]:
# Seleccionar las imagenes que tengan una nubosidad por debajo de 10
ids_noClouds = sentinel_df.img_id[sentinel_df.cloud_cover < 10]
len(ids_noClouds) # 23 Imagenes 

In [ ]:
# Definir region de interes
roi = poi.buffer(10000) # Metros = 10 km

In [ ]:
# Nuevos parametros
parameters = {
    "min":7000,
    "max":16000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B2", "B3", "B4"], # (R, G, B)
    "region":roi # Region de interes
}

In [ ]:
display_img([0, 2], parameters)

In [ ]:
# Imagenes con zoom en la ciudad de Ags
display_img(ids_noClouds, parameters = parameters)

## NDVI

$NDVI = \frac{NIR - RED}{NIR + RED}$

* Saludable = Valores altos
* No saludable = Valores bajos

`ndvi = image.normalizedDifference(["B5", "B4"])`

In [ ]:
# Parametros para el analisis NDVI
palette = ["red", "yellow", "green"]

ndvi_parameters = {
    "min":0, # Este color sera rojo - no saludable
    "max":0.4, # Lo que sea mayor sera verde - saludable
    "dimensions":512,
    "palette":palette,
    "region":roi
}

In [ ]:
# NDVI index por imagen
display_img(ids_noClouds, ndvi_parameters, ndvi_display = True)